In [2]:
import pandas as pd
import sqlite3
import os
from glob import glob
import re

W sumie mamy tutaj trzy tabele:
 - 1_2 -> Zdarzenia wg rodzaju i wielkości w rozbiciu na województwo, powiat, gminę
 - 1a -> Zdarzenia wg rodzaju i wielkości w rozbiciu na komendy wojewódzkie PSP
 - 2 -> Zdarzenia wg rodzaju i wielkości w rozbiciu na miesiące

Zdarzenia wg rodzaju i wielkości w rozbiciu na województwo, powiat, gminę

In [6]:
conn = sqlite3.connect("interwencje_psp.db")
cursor = conn.cursor()

cursor.execute("""
DROP TABLE IF EXISTS "1_2"
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS "1_2" (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    zdarzenie TEXT,
    wojewodztwo TEXT,
    powiat TEXT,
    gmina TEXT,
    kategoria TEXT,
    liczba INTEGER,
    rok INTEGER
)
""")

In [7]:
conn = sqlite3.connect("interwencje_psp.db")

def process_old_format(df_raw, year):
    """Przetwarzanie starego formatu (2016-2022)"""
    event_row = df_raw.iloc[3]
    category_row = df_raw.iloc[4]

    event_types = {}
    current_event = None
    for col in range(2, len(event_row)):
        val = event_row[col]
        if isinstance(val, str):
            current_event = val.strip()
        event_types[col] = current_event

    pozar_cols = [col for col, et in event_types.items() if et and "Pożar" in et]
    mz_cols = [col for col, et in event_types.items() if et and "Miejscowe" in et]
    af_cols = [col for col, et in event_types.items() if et and "Alarm" in et]

    records = []
    current_wojewodztwo = None
    current_powiat = None
    current_gmina = None

    for _, row in df_raw.iloc[6:].iterrows():
        cell = row[1]
        
        if isinstance(cell, str) and cell.isupper():
            current_wojewodztwo = cell.strip()
            current_powiat = None
            current_gmina = None
            continue
        
        if (isinstance(cell, str) and cell[0].isupper() 
            and not cell.isupper() and not cell.islower() 
            and any(c.islower() for c in cell[1:])):  
            current_gmina = cell.strip()
            continue
        
        if pd.isna(cell) or current_wojewodztwo is None:
            continue

        current_powiat = cell.strip()

        # Pożar
        for col in pozar_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(p/", "").replace(")", "").strip()
            if kat_clean.endswith(" m"):
                kat_clean = kat_clean[:-2]
            elif kat_clean.endswith(" ś"):
                kat_clean = kat_clean[:-2]
            elif kat_clean.endswith(" d"):
                kat_clean = kat_clean[:-2]
            elif kat_clean.endswith(" bd"):
                kat_clean = kat_clean[:-3]
            
            value = row[col]
            if pd.notna(value):
                records.append([
                    "Pożar", current_wojewodztwo, current_powiat, current_gmina, kat_clean, int(value), year
                ])

        # Miejscowe zagrożenie 
        for col in mz_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(mz/", "").replace(")", "").strip()
            if "gigantyczne" in kat_clean or "klęska" in kat_clean:
                kat_clean = "gigantyczne lub klęska żywiołowa"
            else:
                if kat_clean.endswith(" m"):
                    kat_clean = kat_clean[:-2]
                elif kat_clean.endswith(" l"):
                    kat_clean = kat_clean[:-2]
                elif kat_clean.endswith(" ś"):
                    kat_clean = kat_clean[:-2]
                elif kat_clean.endswith(" d"):
                    kat_clean = kat_clean[:-2]
            
            value = row[col]
            if pd.notna(value):
                records.append([
                    "Miejscowe zagrożenie", current_wojewodztwo, current_powiat, current_gmina, kat_clean, int(value), year
                ])

        # Alarm fałszywy
        for col in af_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(a/", "").replace(")", "").strip()
            if kat_clean.endswith(" z"):
                kat_clean = kat_clean[:-2]
            elif kat_clean.endswith(" dw"):
                kat_clean = kat_clean[:-3]
            elif kat_clean.endswith(" iw"):
                kat_clean = kat_clean[:-3]
            
            value = row[col]
            if pd.notna(value):
                records.append([
                    "Alarm fałszywy", current_wojewodztwo, current_powiat, current_gmina, kat_clean, int(value), year
                ])

    return records

def process_new_format(df_raw, year):
    """Przetwarzanie nowego formatu (2023-2024) z mapowaniem na stare kategorie"""
    records = []
    
    category_mapping = {
        # Pożary
        "Mały (P/M)": "mały",
        "Średni (P/Ś)": "średni", 
        "Duży (P/D)": "duży",
        "Bardzo duży (P/BD)": "bardzo duży",
        
        # Miejscowe zagrożenia
        "Małe (MZ/M)": "małe",
        "Lokalne (MZ/L)": "lokalne",
        "Średnie (MZ/Ś)": "średnie",
        "Duże (MZ/D)": "duże",
        "Gigantyczne lub klęska żywiołowa (MZ/K)": "gigantyczne lub klęska żywiołowa",
        
        # Alarmy fałszywe
        "Złośliwy (A/Z)": "złośliwy",
        "W dobrej wierze (A/DW)": "w dobrej wierze",
        "Z instalacji wykrywania (A/IW)": "z instalacji wykrywania"
    }
    
    pozar_cols = ["Mały (P/M)", "Średni (P/Ś)", "Duży (P/D)", "Bardzo duży (P/BD)"]
    mz_cols = ["Małe (MZ/M)", "Lokalne (MZ/L)", "Średnie (MZ/Ś)", "Duże (MZ/D)", "Gigantyczne lub klęska żywiołowa (MZ/K)"]
    af_cols = ["Złośliwy (A/Z)", "W dobrej wierze (A/DW)", "Z instalacji wykrywania (A/IW)"]
    
    for _, row in df_raw.iterrows():
        wojewodztwo = row["Województwo"]
        powiat = row["Powiat"] 
        gmina = row["Gmina"]
        
        # Pożary
        for col_name in pozar_cols:
            if col_name in row and pd.notna(row[col_name]) and row[col_name] > 0:
                mapped_category = category_mapping[col_name]
                records.append([
                    "Pożar", wojewodztwo, powiat, gmina, mapped_category, int(row[col_name]), year
                ])
        
        # Miejscowe zagrożenia
        for col_name in mz_cols:
            if col_name in row and pd.notna(row[col_name]) and row[col_name] > 0:
                mapped_category = category_mapping[col_name]
                records.append([
                    "Miejscowe zagrożenie", wojewodztwo, powiat, gmina, mapped_category, int(row[col_name]), year
                ])
        
        # Alarmy fałszywe
        for col_name in af_cols:
            if col_name in row and pd.notna(row[col_name]) and row[col_name] > 0:
                mapped_category = category_mapping[col_name]
                records.append([
                    "Alarm fałszywy", wojewodztwo, powiat, gmina, mapped_category, int(row[col_name]), year
                ])
    
    return records

for year in range(2016, 2025):
    folder = os.path.join("../raw_data", str(year))
    
    if year == 2023:
        patterns = [os.path.join(folder, "1_2023.xlsx")]
    elif year == 2024:
        patterns = [os.path.join(folder, "1.xlsx")]
    else:
        patterns = [
            os.path.join(folder, "1_2_*.xlsx"), 
            os.path.join(folder, "1_2.xlsx"),    
            os.path.join(folder, "1_2*.xlsx")  
        ]
    
    matches = []
    for pattern in patterns:
        matches.extend(glob(pattern))
        if matches:  
            break
    
    if not matches:
        print(f"Brak odpowiedniego pliku w folderze {year}, pomijam.")
        continue

    file_path = matches[0]
    print(f"Przetwarzanie pliku: {file_path}")

    df_raw = pd.read_excel(file_path, header=None if year < 2023 else 0)
    
    if year >= 2023:
        if "Województwo" in df_raw.columns:
            print(f"Rok {year}: wykryto nowy format danych")
            records = process_new_format(df_raw, year)
        else:
            print(f"Rok {year}: nierozpoznany format, próbuję stary format")
            df_raw = pd.read_excel(file_path, header=None)
            records = process_old_format(df_raw, year)
    else:
        records = process_old_format(df_raw, year)

    if records:
        df_final = pd.DataFrame(records, columns=["zdarzenie", "wojewodztwo", "powiat", "gmina", "kategoria", "liczba", "rok"])
        df_final.to_sql("1_2", conn, if_exists="append", index=False)
        print(f"Dodano {len(df_final)} rekordów z roku {year} do tabeli '1_2'.\n")
    else:
        print(f"Rok {year}: brak rekordów do zapisania — możliwe inne formatowanie pliku.\n")

conn.commit()
conn.close()

print("Wszystkie dostępne dane zostały zapisane do bazy interwencje_psp.db.")

Przetwarzanie pliku: ../raw_data\2016\1_2_2016.xlsx
Dodano 3768 rekordów z roku 2016 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2017\1_2_2017.xlsx
Dodano 3768 rekordów z roku 2017 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2018\1_2. Zdarzenia wg rodzaju i wielkości w rozbiciu na województwo, powiat, gminę.xlsx
Dodano 3768 rekordów z roku 2018 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2019\1_2_Zdarzenia wg rodzaju i wielkości w rozbiciu na województwo, powiat, gminę.xlsx
Dodano 3768 rekordów z roku 2019 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2020\1_2_Zdarzenia wg rodzaju i wielkoÿci w rozbiciu na województwo, powiat, gmin®.xlsx
Dodano 3768 rekordów z roku 2020 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2021\1_2_2021.xlsx
Dodano 3768 rekordów z roku 2021 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2022\1_2_2022.xlsx
Dodano 3768 rekordów z roku 2022 do tabeli '1_2'.

Przetwarzanie pliku: ../raw_data\2023\1_2023.xlsx
Rok 2023: wykryto now

In [9]:
conn = sqlite3.connect("interwencje_psp.db")
# df = pd.read_sql_query('SELECT DISTINCT rok FROM "1_2" ORDER BY rok ASC', conn)
df = pd.read_sql_query('SELECT DISTINCT powiat FROM "1_2"', conn)
print(df.head(100))
conn.close()

            powiat
0    bolesławiecki
1   dzierżoniowski
2        głogowski
3         górowski
4         jaworski
..             ...
95        zgierski
96       bocheński
97         brzeski
98     chrzanowski
99       dąbrowski

[100 rows x 1 columns]


1a. Zdarzenia wg rodzaju i wielkości w rozbiciu na komendy wojewódzkie PSP

In [10]:
conn = sqlite3.connect("interwencje_psp.db")
cursor = conn.cursor()

cursor.execute("""
DROP TABLE IF EXISTS "1a"
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS "1a" (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    zdarzenie TEXT,
    glowna_komenda TEXT,
    komenda TEXT,
    kategoria TEXT,
    liczba INTEGER,
    rok INTEGER
)
""")

In [29]:
conn = sqlite3.connect("interwencje_psp.db")

def clean_komenda_name(name):
    """Usuwa kody numeryczne z nazw komend"""
    if pd.isna(name):
        return None
    name = str(name).strip()
    cleaned = re.sub(r'^\d+\s*-\s*', '', name)
    return cleaned.strip()

def process_old_format(df_raw, year):
    """Przetwarzanie starego formatu (2016-2022) — wersja dla komend wojewódzkich PSP"""
    event_row = df_raw.iloc[3]
    category_row = df_raw.iloc[4]

    event_types = {}
    current_event = None
    for col in range(2, len(event_row)):
        val = event_row[col]
        if isinstance(val, str):
            current_event = val.strip()
        event_types[col] = current_event

    pozar_cols = [col for col, et in event_types.items() if et and "Pożar" in et]
    mz_cols = [col for col, et in event_types.items() if et and "Miejscowe" in et]
    af_cols = [col for col, et in event_types.items() if et and "Alarm" in et]

    records = []

    for _, row in df_raw.iloc[6:].iterrows():
        glowna_komenda = row[1]
        if pd.isna(glowna_komenda) or not isinstance(glowna_komenda, str):
            continue
        glowna_komenda = clean_komenda_name(glowna_komenda) 

        # Pożar
        for col in pozar_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(p/", "").replace(")", "").strip()
            kat_clean = re.sub(r"\s*(m|ś|d|bd)$", "", kat_clean)

            value = row[col]
            if pd.notna(value):
                records.append(["Pożar", glowna_komenda, None, kat_clean, int(value), year])

        # Miejscowe zagrożenie
        for col in mz_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(mz/", "").replace(")", "").strip()
            if "gigantyczne" in kat_clean or "klęska" in kat_clean:
                kat_clean = "gigantyczne lub klęska żywiołowa"
            else:
                kat_clean = re.sub(r"\s*(m|l|ś|d)$", "", kat_clean)

            value = row[col]
            if pd.notna(value):
                records.append(["Miejscowe zagrożenie", glowna_komenda, None, kat_clean, int(value), year])

        # Alarm fałszywy
        for col in af_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in kat.upper():
                continue
            kat_clean = kat.lower().replace("(a/", "").replace(")", "").strip()
            kat_clean = re.sub(r"\s*(z|dw|iw)$", "", kat_clean)

            value = row[col]
            if pd.notna(value):
                records.append(["Alarm fałszywy", glowna_komenda, None, kat_clean, int(value), year])

    return records

def process_new_format(df_raw, year):
    """Przetwarzanie nowego formatu (2023–2024) — dane dla KP/M i KW"""
    records = []

    category_mapping = {
        "Mały (P/M)": "mały", "Średni (P/Ś)": "średni", "Duży (P/D)": "duży", "Bardzo duży (P/BD)": "bardzo duży",
        "Małe (MZ/M)": "małe", "Lokalne (MZ/L)": "lokalne", "Średnie (MZ/Ś)": "średnie",
        "Duże (MZ/D)": "duże", "Gigantyczne lub klęska żywiołowa (MZ/K)": "gigantyczne lub klęska żywiołowa",
        "Złośliwy (A/Z)": "złośliwy", "W dobrej wierze (A/DW)": "w dobrej wierze",
        "Z instalacji wykrywania (A/IW)": "z instalacji wykrywania"
    }

    pozar_cols = [col for col in category_mapping if "(P/" in col]
    mz_cols = [col for col in category_mapping if "(MZ/" in col]
    af_cols = [col for col in category_mapping if "(A/" in col]

    for _, row in df_raw.iterrows():
        komenda = row.get("Nazwa KP/M PSP")
        glowna_komenda = row.get("Nazwa KW PSP")

        if pd.isna(komenda):
            continue
        
        komenda = clean_komenda_name(komenda)  
        glowna_komenda = clean_komenda_name(glowna_komenda)  

        for col in pozar_cols:
            if col in row and pd.notna(row[col]) and row[col] > 0:
                records.append(["Pożar", glowna_komenda, komenda, category_mapping[col], int(row[col]), year])

        for col in mz_cols:
            if col in row and pd.notna(row[col]) and row[col] > 0:
                records.append(["Miejscowe zagrożenie", glowna_komenda, komenda, category_mapping[col], int(row[col]), year])

        for col in af_cols:
            if col in row and pd.notna(row[col]) and row[col] > 0:
                records.append(["Alarm fałszywy", glowna_komenda, komenda, category_mapping[col], int(row[col]), year])

    return records

# --------------------------------------------
for year in range(2016, 2025):
    folder = os.path.join("../raw_data", str(year))

    if year == 2023:
        patterns = [os.path.join(folder, "1a_2023.xlsx")]
    elif year == 2024:
        patterns = [os.path.join(folder, "1a.xlsx")]
    else:
        patterns = [os.path.join(folder, "1a*.xlsx")]

    matches = []
    for pattern in patterns:
        matches.extend(glob(pattern))
        if matches:
            break

    if not matches:
        print(f"Brak odpowiedniego pliku w folderze {year}, pomijam.")
        continue

    file_path = matches[0]
    print(f"Przetwarzanie pliku: {file_path}")

    df_raw = None
    for header_row in [0, 1, 2, 3, 4]:
        df_try = pd.read_excel(file_path, header=header_row)
        if any("Nazwa KP/M PSP" in str(col) for col in df_try.columns):
            df_raw = df_try
            print(f"Rok {year}: wykryto nowy format danych (nagłówki w wierszu {header_row})")
            break

    if df_raw is not None:
        records = process_new_format(df_raw, year)
    else:
        print(f"Rok {year}: nierozpoznany format, próbuję stary format")
        df_raw = pd.read_excel(file_path, header=None)
        records = process_old_format(df_raw, year)

    if records:
        df_final = pd.DataFrame(records, columns=["zdarzenie", "glowna_komenda", "komenda", "kategoria", "liczba", "rok"])
        df_final.to_sql("1a", conn, if_exists="append", index=False)
        print(f"Dodano {len(df_final)} rekordów z roku {year} do tabeli '1a'.\n")
    else:
        print(f"Rok {year}: brak rekordów do zapisania — możliwe inne formatowanie pliku.\n")

conn.commit()
conn.close()

print("Wszystkie dostępne dane zostały zapisane do bazy interwencje_psp.db.")

Przetwarzanie pliku: ../raw_data\2016\1a_2016.xlsx
Rok 2016: nierozpoznany format, próbuję stary format
Dodano 192 rekordów z roku 2016 do tabeli '1a'.

Przetwarzanie pliku: ../raw_data\2017\1a_2017.xlsx
Rok 2017: nierozpoznany format, próbuję stary format
Dodano 192 rekordów z roku 2017 do tabeli '1a'.

Przetwarzanie pliku: ../raw_data\2018\1a. Zdarzenia wg rodzaju i wielkości w rozbiciu na komendy wojewódzkie PSP.xlsx
Rok 2018: nierozpoznany format, próbuję stary format
Dodano 192 rekordów z roku 2018 do tabeli '1a'.

Przetwarzanie pliku: ../raw_data\2019\1a_Zdarzenia wg rodzaju i wielkości w rozbiciu na komendy wojewódzkie PSP.xlsx
Rok 2019: nierozpoznany format, próbuję stary format
Dodano 192 rekordów z roku 2019 do tabeli '1a'.

Przetwarzanie pliku: ../raw_data\2020\1a_Zdarzenia wg rodzaju i wielkoÿci w rozbiciu na komendy wojewódzkie PSP.xlsx
Rok 2020: nierozpoznany format, próbuję stary format
Dodano 192 rekordów z roku 2020 do tabeli '1a'.

Przetwarzanie pliku: ../raw_data\202

In [30]:
conn = sqlite3.connect("interwencje_psp.db")
# df = pd.read_sql_query('SELECT DISTINCT rok FROM "1_2" ORDER BY rok ASC', conn)
df = pd.read_sql_query('SELECT DISTINCT glowna_komenda FROM "1a"', conn)
print(df.head(100))
conn.close()

                glowna_komenda
0               KW PSP Wrocław
1                 KW PSP Toruń
2                KW PSP Lublin
3   KW PSP Gorzów Wielkopolski
4                  KW PSP Łódź
5                KW PSP Kraków
6              KW PSP Warszawa
7                 KW PSP Opole
8               KW PSP Rzeszów
9             KW PSP Białystok
10               KW PSP Gdańsk
11             KW PSP Katowice
12               KW PSP Kielce
13              KW PSP Olsztyn
14               KW PSP Poznań
15             KW PSP Szczecin


2. Zdarzenia wg rodzaju i wielkości w rozbiciu na miesiące

In [35]:
conn = sqlite3.connect("interwencje_psp.db")
cursor = conn.cursor()

cursor.execute("""
DROP TABLE IF EXISTS "2"
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS "2" (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    zdarzenie TEXT,
    miesiac TEXT,
    kategoria TEXT,
    liczba INTEGER,
    rok INTEGER
)
""")

In [36]:
conn = sqlite3.connect("interwencje_psp.db")

def process_old_format_monthly(df_raw, year):
    """Przetwarzanie starego formatu (2016–2022) z podziałem na miesiące"""
    event_row = df_raw.iloc[3]
    category_row = df_raw.iloc[4]

    event_types = {}
    current_event = None
    for col in range(2, len(event_row)):
        val = event_row[col]
        if isinstance(val, str):
            current_event = val.strip()
        event_types[col] = current_event

    pozar_cols = [col for col, et in event_types.items() if et and "Pożar" in et]
    mz_cols = [col for col, et in event_types.items() if et and "Miejscowe" in et]
    af_cols = [col for col, et in event_types.items() if et and "Alarm" in et]

    records = []

    for _, row in df_raw.iloc[6:].iterrows():
        cell = row[1]

        if pd.isna(cell):
            continue

        miesiac = str(cell).strip().capitalize()
        if not miesiac or miesiac.upper() in {"RAZEM", "OGÓŁEM"}:
            continue

        # Pożary
        for col in pozar_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in str(kat).upper():
                continue
            kat_clean = kat.lower().replace("(p/", "").replace(")", "").strip()
            kat_clean = kat_clean.removesuffix(" m").removesuffix(" ś").removesuffix(" d").removesuffix(" bd")

            value = row[col]
            if pd.notna(value):
                records.append(["Pożar", miesiac, kat_clean, int(value), year])

        # Miejscowe zagrożenia
        for col in mz_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in str(kat).upper():
                continue
            kat_clean = kat.lower().replace("(mz/", "").replace(")", "").strip()
            if "gigantyczne" in kat_clean or "klęska" in kat_clean:
                kat_clean = "gigantyczne lub klęska żywiołowa"
            else:
                kat_clean = kat_clean.removesuffix(" m").removesuffix(" ś").removesuffix(" d").removesuffix(" l")

            value = row[col]
            if pd.notna(value):
                records.append(["Miejscowe zagrożenie", miesiac, kat_clean, int(value), year])

        # Alarmy fałszywe
        for col in af_cols:
            kat = category_row[col]
            if pd.isna(kat) or "RAZEM" in str(kat).upper():
                continue
            kat_clean = kat.lower().replace("(a/", "").replace(")", "").strip()
            kat_clean = kat_clean.removesuffix(" z").removesuffix(" dw").removesuffix(" iw")

            value = row[col]
            if pd.notna(value):
                records.append(["Alarm fałszywy", miesiac, kat_clean, int(value), year])

    return records

def process_new_format_monthly(df_raw, year):
    """Przetwarzanie nowego formatu (2023–2024) z podziałem na miesiące"""
    
    if "Miesiące" not in df_raw.columns:
        return []

    category_mapping = {
        # Pożary
        "Mały (P/M)": "mały",
        "Średni (P/Ś)": "średni", 
        "Duży (P/D)": "duży",
        "Bardzo duży (P/BD)": "bardzo duży",

        # Miejscowe zagrożenia
        "Małe (MZ/M)": "małe",
        "Lokalne (MZ/L)": "lokalne",
        "Średnie (MZ/Ś)": "średnie",
        "Duże (MZ/D)": "duże",
        "Gigantyczne lub klęska żywiołowa (MZ/K)": "gigantyczne lub klęska żywiołowa",

        # Alarmy fałszywe
        "Złośliwy (A/Z)": "złośliwy",
        "W dobrej wierze (A/DW)": "w dobrej wierze",
        "Z instalacji wykrywania (A/IW)": "z instalacji wykrywania"
    }

    records = []

    for _, row in df_raw.iterrows():
        if not isinstance(row["Miesiące"], str) or row["Miesiące"].upper() in ["RAZEM", "OGÓŁEM"]:
            continue

        month = row["Miesiące"].strip().capitalize()

        # Pożary
        for size_col in ["Mały (P/M)", "Średni (P/Ś)", "Duży (P/D)", "Bardzo duży (P/BD)"]:
            if size_col in df_raw.columns:
                value = row[size_col]
                if pd.notna(value) and value > 0:
                    records.append([
                        "Pożar", month, category_mapping[size_col], int(value), year
                    ])

        # Miejscowe zagrożenia
        for mz_col in ["Małe (MZ/M)", "Lokalne (MZ/L)", "Średnie (MZ/Ś)", "Duże (MZ/D)", "Gigantyczne lub klęska żywiołowa (MZ/K)"]:
            if mz_col in df_raw.columns:
                value = row[mz_col]
                if pd.notna(value) and value > 0:
                    records.append([
                        "Miejscowe zagrożenie", month, category_mapping[mz_col], int(value), year
                    ])

        # Alarmy fałszywe
        for af_col in ["Złośliwy (A/Z)", "W dobrej wierze (A/DW)", "Z instalacji wykrywania (A/IW)"]:
            if af_col in df_raw.columns:
                value = row[af_col]
                if pd.notna(value) and value > 0:
                    records.append([
                        "Alarm fałszywy", month, category_mapping[af_col], int(value), year
                    ])

    return records


for year in range(2016, 2025):
    folder = os.path.join("../raw_data", str(year))
    
    if year == 2023:
        patterns = [os.path.join(folder, "2_2023.xlsx")]
    elif year == 2024:
        patterns = [os.path.join(folder, "2.xlsx")]
    else:
        patterns = [os.path.join(folder, "2_*.xlsx")]

    matches = []
    for pattern in patterns:
        matches.extend(glob(pattern))
        if matches:
            break

    if not matches:
        print(f"Brak odpowiedniego pliku w folderze {year}, pomijam.")
        continue

    file_path = matches[0]
    print(f"Przetwarzanie pliku: {file_path}")

    try:
        if year >= 2023:
            df_raw = pd.read_excel(file_path, header=0)
            if "Miesiące" in df_raw.columns:
                print(f"Rok {year}: wykryto nowy format danych (miesięczny)")
                records = process_new_format_monthly(df_raw, year)
            else:
                print(f"Rok {year}: nierozpoznany format, próbuję stary format")
                df_raw = pd.read_excel(file_path, header=None)
                records = process_old_format_monthly(df_raw, year)
        else:
            df_raw = pd.read_excel(file_path, header=None)
            records = process_old_format_monthly(df_raw, year)
    except Exception as e:
        print(f"Błąd podczas przetwarzania pliku {file_path}: {str(e)}")
        continue

    if records:
        df_final = pd.DataFrame(records, columns=["zdarzenie", "miesiac", "kategoria", "liczba", "rok"])
        df_final.to_sql("2", conn, if_exists="append", index=False)
        print(f"Dodano {len(df_final)} rekordów z roku {year} do tabeli '2'.\n")
    else:
        print(f"Rok {year}: brak rekordów do zapisania — możliwe inne formatowanie pliku.\n")

conn.commit()
conn.close()

print("Wszystkie dostępne dane zostały zapisane do bazy interwencje_psp.db.")

Przetwarzanie pliku: ../raw_data\2016\2_2016.xlsx
Dodano 144 rekordów z roku 2016 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2017\2_2017.xlsx
Dodano 144 rekordów z roku 2017 do tabeli '2'.

Brak odpowiedniego pliku w folderze 2018, pomijam.
Przetwarzanie pliku: ../raw_data\2019\2_Zdarzenia wg rodzaju i wielkości w rozbiciu na miesiące.xlsx
Dodano 144 rekordów z roku 2019 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2020\2_Zdarzenia wg rodzaju i wielkoÿci w rozbiciu na miesiÑce.xlsx
Dodano 144 rekordów z roku 2020 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2021\2_2021.xlsx
Dodano 144 rekordów z roku 2021 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2022\2_2022.xlsx
Dodano 144 rekordów z roku 2022 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2023\2_2023.xlsx
Rok 2023: wykryto nowy format danych (miesięczny)
Dodano 143 rekordów z roku 2023 do tabeli '2'.

Przetwarzanie pliku: ../raw_data\2024\2.xlsx
Rok 2024: wykryto nowy format danych (miesięczny)
Dodano 146 rekor

In [37]:
conn = sqlite3.connect("interwencje_psp.db")
# df = pd.read_sql_query('SELECT DISTINCT rok FROM "1_2" ORDER BY rok ASC', conn)
df = pd.read_sql_query('SELECT * FROM "2"', conn)
print(df.head(100))
conn.close()

     id             zdarzenie      miesiac                kategoria  liczba  \
0     1                 Pożar         Luty                     mały    8070   
1     2                 Pożar         Luty                   średni     288   
2     3                 Pożar         Luty                     duży      24   
3     4                 Pożar         Luty              bardzo duży       3   
4     5  Miejscowe zagrożenie         Luty                     małe    2399   
..  ...                   ...          ...                      ...     ...   
95   96        Alarm fałszywy     Wrzesień  z instalacji wykrywania    1490   
96   97                 Pożar  Październik                     mały    6255   
97   98                 Pożar  Październik                   średni     238   
98   99                 Pożar  Październik                     duży      21   
99  100                 Pożar  Październik              bardzo duży       9   

     rok  
0   2016  
1   2016  
2   2016  
3   201